In [ ]:
# Standard data manipulation libraries
import pandas as pd
import numpy as np

# Webscraping imports
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
# Test with random sample of 1000 ssvids
sample_ssvid = pd.read_csv('./data/sample_ssvid.csv')

In [ ]:
# convert ssvid column to list of numbers
ssvids = sample_ssvid['ssvid'].values.tolist()
ssvids[:10]

In [ ]:
# define chromedriver location
chromedriver = '/Users/jessicaertel/Downloads/chromedriver'

# define driver object
driver = webdriver.Chrome(chromedriver)

# Open browser
driver.get('https://webaccounts.imo.org')

# Select 'Public' in drop down menu
authority = Select(driver.find_element_by_name('ctl00$cpMain$ddlAuthorityType'))
authority.select_by_value('PUBLIC')

# Enter username and password
driver.find_element_by_id("ctl00_cpMain_txtUsername").send_keys("jessicaertel")
driver.find_element_by_id("ctl00_cpMain_txtPassword").send_keys("fishRfriends0")

# Click log in button
driver.find_element_by_id("ctl00_cpMain_btnLogin").click()

# Navigate to Global Integrated Shipping Information System
driver.find_element_by_id("ctl00_cpMain_lnkGISIS").click()
driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr/td/div[3]/a/div[2]/span[1]').click()

# Create lists to hold scraped data
name, imo_num, flag, call_sign, mmsi, vessel_type, built, tonnage, owner = [],[],[],[],[],[],[],[],[]
        
# For each number in ssvids
for num in ssvids:
    
    # Enter number into MMSI field and click search
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").send_keys(num)
    driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_btnSearchShips").click()
    driver.implicitly_wait(10)
    #sleep(5)
    
    # Define two outcomes from search
    no_result = EC.presence_of_element_located((By.XPATH, "/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr/td/p/a"))
    result = EC.presence_of_element_located((By.XPATH, "/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr[1]"))

#     no_result = driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/tbody/tr')
#     result = driver.find_element_by_xpath('/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[1]/td/center/table/tbody/tr[3]/td[1]/div/table/thead/tr')
#     result = driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_gridShips")
    
    
    # if no search result is displayed, remove text and add the next number
    if no_result:
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

    # if there is a search result click on it
    elif result: 
        driver.find_element_by_class_name('gridviewer_row tipsterise noline tooltipstered').click()
        
        # scrape and append data to each list
        name.append(driver.find_element_by_id("sNameCur"))
        imo_num.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[2]/td[2]"))
        flag.append(driver.find_element_by_id("sFlagCur"))
        call_sign.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[4]/td[2]"))
        mmsi.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr[5]/td[2]"))
        vessel_type.append(driver.find_element_by_id("sTypeCur"))
        built.append(driver.find_element_by_id("sBuildDateCur"))
        tonnage.append(driver.find_element_by_xpath("/html/body/form/table[4]/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr[3]/td[2]"))
        owner.append(driver.find_element_by_id("sRegOwnerCur"))
        
        # return to the search page and clear search
        driver.execute_script("window.history.go(-1)")
        driver.find_element_by_id("ctl00_bodyPlaceHolder_Default_tbxShipMMSI").clear()

In [ ]:
# transform lists into pandas dataframe
imo = pd.DataFrame({'name': name,
                     'imo_num': imo_num,
                     'flag': flag,
                     'call_sign': call_sign,
                     'mmsi': mmsi,
                     'vessel_type': vessel_type,
                     'built' : built,
                     'tonnage': tonnage,
                     'owner': owner})

## References
- https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
- https://selenium-python.readthedocs.io/locating-elements.html
- https://stackoverflow.com/questions/27338742/how-do-i-send-a-delete-keystroke-to-a-text-field-using-selenium-with-python